In [0]:
!pip install glove_python

In [0]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.sentiment import sentiment_analyzer
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
from glove import Glove

In [0]:
nltk.download('vader_lexicon')

In [0]:
data=pd.read_csv("https://raw.githubusercontent.com/zfz/twitter_corpus/master/full-corpus.csv")
url = "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Office_Products_5.json.gz"
r = pd.read_json(url,lines=True)
df1 = r.copy()
twitr = data.copy() 

In [0]:
#df1['sentiR']=df1['reviewText'].apply(lambda x:(TextBlob(x).sentiment)[0])
twitr['sentiT']=twitr['TweetText'].apply(lambda x:(TextBlob(x).sentiment)[0])

In [0]:
sid = SentimentIntensityAnalyzer()
def sentIntens(text):
  compctr,compound =0,0
  for sentence in text.split('.'):
    ss = sid.polarity_scores(sentence)####Logic is a bit shaky after this 
    if float(ss['compound'])==0.000:
      compctr +=1
    else:
      compound +=ss['compound']
  if compctr == text.count('.'):
    return 'irrelevant'
  elif compound > 0.05:
    return 'positive'
  elif compound < -0.05:
    return 'negative'
  else:
    return 'neutral'

In [0]:
df1['sentIntensS'] = df1['summary'].apply(lambda x:sentIntens(x))

In [0]:
twitr['sentIntensT'] = twitr['TweetText'].apply(lambda x:sentIntens(x))

In [128]:
abc = classification_report(twitr['sentIntensT'],twitr['Sentiment'],output_dict=True)
df = pd.DataFrame(abc).transpose()
accuracy_score(twitr['Sentiment'],twitr['sentIntensT'])

0.3590846860942695

In [123]:
df

,precision,recall,f1-score,support
irrelevant,0.285376,0.223148,0.250455,2160.000000
negative,0.108392,0.632653,0.185075,98.000000
neutral,0.525075,0.464719,0.493057,2636.000000
positive,0.129094,0.305936,0.181572,219.000000
accuracy,0.359085,0.359085,0.359085,0.359085
macro avg,0.261984,0.406614,0.277540,5113.000000
weighted avg,0.398866,0.359085,0.371324,5113.000000


In [129]:
twitr

,Topic,Sentiment,TweetId,TweetDate,TweetText,sentiT,SentimentC,sentIntensT
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...,0.000000,1,neutral
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...,0.500000,1,irrelevant
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...,0.393750,1,positive
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...,0.541667,1,irrelevant
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...,0.200000,1,irrelevant
...,...,...,...,...,...,...,...,...
5108,twitter,irrelevant,126855687060987904,Thu Oct 20 03:02:07 +0000 2011,me re copè con #twitter,0.000000,-2,neutral
5109,twitter,irrelevant,126855171702661120,Thu Oct 20 03:00:04 +0000 2011,Buenas noches genteeee :) #twitter los quieroo...,0.500000,-2,irrelevant
5110,twitter,irrelevant,126854999442587648,Thu Oct 20 02:59:23 +0000 2011,#twitter tiene la mala costumbre de ponerce bn...,-0.250000,-2,irrelevant
5111,twitter,irrelevant,126854818101858304,Thu Oct 20 02:58:40 +0000 2011,Oi @flaviasansi. Muito bem vinda ao meu #Twitt...,0.000000,-2,neutral
